# Setup

In [1]:
import pandas as pd

In [2]:
original = pd.read_csv("./data/original.csv")

In [3]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5571 entries, 0 to 5570
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   RC#          5570 non-null   float64
 1   RC Name      5570 non-null   object 
 2   Category     5570 non-null   float64
 3   Vendor#      5570 non-null   float64
 4   Vendor Name  5570 non-null   object 
 5   Address 1    5546 non-null   object 
 6   Address 2    1290 non-null   object 
 7   City         5471 non-null   object 
 8   State        5373 non-null   object 
 9   ZIP code     5559 non-null   object 
 10  Amount       5571 non-null   int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 478.9+ KB


In [4]:
updated = pd.read_csv("./data/updated.csv")

In [5]:
updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5921 entries, 0 to 5920
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   RC#          5920 non-null   float64
 1   RC Name      5920 non-null   object 
 2   Category     5920 non-null   float64
 3   Vendor#      5920 non-null   float64
 4   Vendor Name  5920 non-null   object 
 5   Address 1    5896 non-null   object 
 6   Address 2    1359 non-null   object 
 7   City         5821 non-null   object 
 8   State        5716 non-null   object 
 9   ZIP code     5909 non-null   object 
 10  Amount       5921 non-null   int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 509.0+ KB
